# **What are .nii files?**
NII format is created by Neuroimaging Informatics Technology Initiative. It is commonly used to store magnetic resonance imaging (MRI) data.

This sections of the code is just to see how to open and visualize this .nii data in python.

In [ ]:
import nibabel as nib
from glob import glob
import matplotlib.pylab as plt

In [ ]:
path = '/content/BraTS20_Training_001_flair.nii'

In [ ]:
img = nib.load(path).get_fdata()
img.shape

In [ ]:
print(f"The .nii files are stored in memory as numpy's: {type(img)}.")

# **Axial slices**
The code here is to visualize the the Axial of the image.

The Axial plane is an anatomical plane that divides the body into x and y plane.

In [ ]:
num_slices = img.shape[2]

In [ ]:
num_columns = 4
num_rows = (num_slices + num_columns - 1) // num_columns


In [ ]:
# Visualize all Axial slices from the start
for i in range(num_slices):
    fig, ax = plt.subplots(figsize=(6, 6))  # Adjust figsize as needed
    ax.imshow(img[:, :, i], cmap='gray', aspect='auto', vmin=0, vmax=255)
    ax.axis('off')
    plt.show()
    print(i)

# **Coronal slices**

The code here is to visualize the the coronal of the image.

The coronal plane is an anatomical plane that divides the body into dorsal and ventral sections. It is perpendicular to the sagittal and transverse planes.

In [ ]:
# Check the shape of the MRI data
print("Shape of MRI data:", img.shape)

# Calculate the middle slice index
num_coronal_slices = img.shape[0] 

In [ ]:
# Visualize all coronal slices from the start
for i in range(num_coronal_slices):
    fig, ax = plt.subplots(figsize=(6, 6))  # Adjust figsize as needed
    ax.imshow(img[i, :, :], cmap='gray', aspect='auto', vmin=0, vmax=255)
    ax.axis('off')
    plt.show()
    print(i)

# **Sagittal slices**
The code here is to visualize the the sagittal of the image.

In [ ]:
# Check the shape of the MRI data
print("Shape of MRI data:", img.shape)

# Determine the number of slices in the second dimension (sagittal plane)
num_sagittal_slices = img.shape[1]

In [ ]:
# Visualize all sagittal slices from the start
for i in range(num_sagittal_slices):
    fig, ax = plt.subplots(figsize=(6, 6))  # Adjust figsize as needed
    ax.imshow(img[:, i, :], cmap='gray', aspect='auto', vmin=0, vmax=255)
    ax.axis('off')
    plt.show()

# **Neural network development**



Mounting my google drive to get the dataset saved in my drive

In [ ]:
import tensorflow as tf
import zipfile
import numpy as np
import random
import os
from tqdm import tqdm
from skimage.io import imread, imshow
from skimage.transform import resize
from sklearn.model_selection import train_test_split
from keras.layers import Conv2D, Dropout, MaxPooling2D, Conv2DTranspose, Concatenate, Lambda, Input
from keras.models import Model
from keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard
from keras.layers import Activation, BatchNormalization
from keras.optimizers import Adam
from keras.metrics import MeanIoU
from keras import backend as K
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# You need to mount your google drive in order to load the data:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
ls

drive/  sample_data/


In [ ]:
import glob

# Define the base paths for training and validation data
training_base_path = "/content/drive/MyDrive/340ProjectData/BraTS2020_TrainingData/MICCAI_BraTS2020_TrainingData"
validation_base_path = "/content/drive/MyDrive/340ProjectData/BraTS2020_ValidationData/MICCAI_BraTS2020_ValidationData"

# Function to get all .nii files for each case
def get_nii_files(base_path):
    case_paths = sorted(glob.glob(os.path.join(base_path, "BraTS20_*")))
    print(f"Found {len(case_paths)} case directories in {base_path}.")  # Debug print
    nii_files = [sorted(glob.glob(os.path.join(case, "*.nii*"))) for case in case_paths]
    return nii_files

training_nii_files = get_nii_files(training_base_path)
validation_nii_files = get_nii_files(validation_base_path)

# Debug print to check if the lists are populated
print(f"Number of training cases found: {len(training_nii_files)}")
print(f"Number of validation cases found: {len(validation_nii_files)}")

# Print the paths of the .nii files for the first case in the training data, if available
if training_nii_files:
    print("Training data files for the first case:")
    for file_path in training_nii_files[0]:
        print(file_path)
else:
    print("No training data files found.")

# Do the same for the validation data, if available
if validation_nii_files:
    print("Validation data files for the first case:")
    for file_path in validation_nii_files[0]:
        print(file_path)
else:
    print("No validation data files found.")

Found 369 case directories in /content/drive/MyDrive/340ProjectData/BraTS2020_TrainingData/MICCAI_BraTS2020_TrainingData.
Found 125 case directories in /content/drive/MyDrive/340ProjectData/BraTS2020_ValidationData/MICCAI_BraTS2020_ValidationData.
Number of training cases found: 369
Number of validation cases found: 125
Training data files for the first case:
/content/drive/MyDrive/340ProjectData/BraTS2020_TrainingData/MICCAI_BraTS2020_TrainingData/BraTS20_Training_001/BraTS20_Training_001_flair.nii
/content/drive/MyDrive/340ProjectData/BraTS2020_TrainingData/MICCAI_BraTS2020_TrainingData/BraTS20_Training_001/BraTS20_Training_001_seg.nii
/content/drive/MyDrive/340ProjectData/BraTS2020_TrainingData/MICCAI_BraTS2020_TrainingData/BraTS20_Training_001/BraTS20_Training_001_t1.nii
/content/drive/MyDrive/340ProjectData/BraTS2020_TrainingData/MICCAI_BraTS2020_TrainingData/BraTS20_Training_001/BraTS20_Training_001_t1ce.nii
/content/drive/MyDrive/340ProjectData/BraTS2020_TrainingData/MICCAI_BraT

In [ ]:
# Set the image dimensions (adjust as needed)
IMG_WIDTH = 240
IMG_HEIGHT = 240
IMG_CHANNELS = 1  # for grayscale 3d images

In [ ]:
# Define the input layer
inputs = Input((IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS))

In [ ]:
import numpy as np
import nibabel as nib
from tensorflow.keras.utils import Sequence
from skimage.transform import resize

class NiiDataGenerator(Sequence):
    'Generates data for Keras'
    def __init__(self, image_paths, mask_paths, batch_size=1, dim=(240, 240), n_channels=1, n_slices=155, shuffle=True):
        'Initialization'
        self.dim = dim
        self.batch_size = batch_size
        self.image_paths = image_paths
        self.mask_paths = mask_paths
        self.n_channels = n_channels
        self.n_slices = n_slices
        self.shuffle = shuffle
        self.on_epoch_end()

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.image_paths) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index * self.batch_size:(index + 1) * self.batch_size]

        # Find list of IDs
        image_paths_temp = [self.image_paths[k] for k in indexes]
        mask_paths_temp = [self.mask_paths[k] for k in indexes]

        # Generate data
        X, y = self.__data_generation(image_paths_temp, mask_paths_temp)

        return X, y

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.image_paths))
        if self.shuffle:
            np.random.shuffle(self.indexes)

    def __data_generation(self, image_paths_temp, mask_paths_temp):
      # Generates data containing batch_size samples
      # Initialization
      X = np.empty((self.batch_size, *self.dim, self.n_channels))
      y = np.empty((self.batch_size, *self.dim, 1))  # Only one channel for the mask

      # Generate data
      for i, (img_paths, mask_path) in enumerate(zip(image_paths_temp, mask_paths_temp)):
          # Load and process the images
          images = []
          for img_path in img_paths:
              img = nib.load(img_path).get_fdata()
              img = resize(img, self.dim, mode='constant', preserve_range=True)
              images.append(img)
          X[i,] = np.stack(images, axis=-1)  # Stack along the channel dimension

          # Load and process the mask
          mask = nib.load(mask_path).get_fdata()
          mask = resize(mask, self.dim, mode='constant', preserve_range=True)
          y[i,] = np.expand_dims(mask, axis=-1)

      return X, y



In [ ]:
# Assuming that 'flair', 't1', 't1ce', and 't2' are your image files and 'seg' is your mask file
image_paths = []  # This will be a list of lists, containing paths to all image modalities for each case
mask_paths = []  # This will contain paths to the segmentation masks for each case
count = 0
for case_files in training_nii_files:
    count = count + 1

    # Find the segmentation mask file, which should have '_seg.nii' in its name
    mask_files = [f for f in case_files if '_seg.nii' in f]
    print(count)
    # Check if a mask file was found
    if mask_files:
        mask_file = mask_files[0]
        mask_paths.append(mask_file)

        # All other files in the case directory are considered image files
        image_files = [f for f in case_files if f != mask_file]
        image_paths.append(image_files)
    else:
        print(f"No segmentation mask found for case: {case_files}")
        # Handle the case where no segmentation mask is found
        # For example, you might want to skip this case or raise an error

# Now you have a list of image file paths and mask file paths
# You can create a data generator instance with these lists
data_generator = NiiDataGenerator(image_paths=image_paths, mask_paths=mask_paths, batch_size=8, dim=(240, 240), n_channels=4)  # n_channels is set to 4 because you have 4 image modalities per case



1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
No segmentation mask found for case: ['/content/drive/MyDrive/340ProjectData/BraTS2020_TrainingData/MICCAI_BraTS2020_TrainingData/BraTS20_Training_070/BraTS20_Training_070_flair.nii']
71
No segmentation mask found for case: []
72
No segmentation mask found for case: []
73
No segmentation mask found for case: []
74
No segmentation mask found for case: []
75
No segmentation mask found for case: []
76
No segmentation mask found for case: []
77
No segmentation mask found for case: []
78
No segmentation mask found for case: []
79
No segmentation mask found for case: []
80
No segmentation mask found for case: []
81
No segmentation mask found for case: []
82
No segmentation mask found for case: []
83
No segmentation mask found for case: []
84
No segmentation mask found for case: []
85
No segment